In [1]:
import os
import pandas as pd
import numpy as np
import time
import gc
import random
import jieba
from tqdm import *
from sklearn.metrics import roc_auc_score,accuracy_score,f1_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold,KFold,train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD,PCA,LatentDirichletAllocation,NMF
from collections import Counter
from gensim.models import Word2Vec
def one_hot_col(col):
    '''标签编码'''
    lbl = preprocessing.LabelEncoder()
    lbl.fit(col)
    return lbl

In [2]:
def cc_label(i):
    if i=='传输系统-传输设备':
        return 0
    elif i=='传输系统-光缆故障':
        return 1
    elif i=='传输系统-其他原因':
        return 2
    elif i=='动力环境-UPS':
        return 3
    elif i=='动力环境-电力部门供电':
        return 4
    elif i=='动力环境-电源线路故障':
        return 5
    elif i=='动力环境-动环监控系统':
        return 6
    elif i=='动力环境-动力环境故障':
        return 7
    elif i=='动力环境-高低压设备':
        return 8
    elif i=='动力环境-环境':
        return 9
    elif i=='动力环境-开关电源':
        return 10
    elif i=='其他-误告警或自动恢复':
        return 11
    elif i=='人为操作-告警测试':
        return 12
    elif i=='人为操作-工程施工':
        return 13
    elif i=='人为操作-物业原因':
        return 14
    elif i=='主设备-参数配置异常':
        return 15
    elif i=='主设备-其他':
        return 16
    elif i=='主设备-软件故障':
        return 17
    elif i=='主设备-设备复位问题':
        return 18
    elif i=='主设备-设备连线故障':
        return 19
    elif i=='主设备-天馈线故障':
        return 20
    elif i=='主设备-信源问题':
        return 21
    elif i=='主设备-硬件故障':
        return 22

In [3]:
train_label= pd.read_csv(open(r'../data/训练故障工单.csv'))
test_label= pd.read_csv(open(r'../data/测试故障工单.csv'))
print(train_label.shape)
train_label=train_label.drop_duplicates(['故障发生时间','涉及告警基站或小区名称',\
                   '故障原因定位（大类）']).reset_index(drop=True) ##去重 （18540，4）
print(train_label.shape)
label=pd.concat([train_label,test_label],axis=0).reset_index(drop=True)
label['故障发生时间'] = pd.to_datetime(label['故障发生时间'])
# label=label[label['故障发生时间']>pd.to_datetime('2019-01-31 23:00:00')]
label=label.sort_values(by=['涉及告警基站或小区名称','故障发生时间']).reset_index(drop=True)

(60346, 4)
(59758, 4)


In [4]:
traing= pd.read_csv(open(r'../data/训练告警.csv'))
testg= pd.read_csv(open(r'../data/测试告警.csv'))
data=pd.concat([traing,testg],axis=0).reset_index(drop=True)
data=data[~data['涉及告警基站或小区名称'].isnull()]
data=data.drop_duplicates().reset_index(drop=True)

data['告警发生时间'] = data['告警发生时间'].apply(lambda x:x.replace('FEB','02'))
data['告警发生时间'] = data['告警发生时间'].apply(lambda x:x.replace('MAR','03'))
data['告警发生时间'] = pd.to_datetime(data['告警发生时间'],format="%d-%m-%Y %H:%M:%S")
lbl = preprocessing.LabelEncoder()
data['告警标题'] = lbl.fit_transform(data['告警标题'])
data['告警标题'] = data['告警标题'].astype(str)

In [5]:
sp_word = ['29','16', '46', '22', '33', '41', '113', '120', '8', '96', '59', '13', '40',
       '39', '26', '34', '37', '32', '94', '65', '14', '97', '129', '42', '15',
       '121', '108', '128', '12', '31', '85', '119', '123']
feat = data[['涉及告警基站或小区名称', '告警标题']].copy()
feat = feat.groupby(['涉及告警基站或小区名称'])['告警标题'].agg(lambda x: ' '.join(x)).reset_index(name='Product_list')
tf_idf = TfidfVectorizer(stop_words=sp_word)
tf_vec = tf_idf.fit_transform(feat['Product_list'].values.tolist())
tf_df = pd.DataFrame(tf_vec.toarray())
tf_df['涉及告警基站或小区名称'] = feat['涉及告警基站或小区名称'].values
tf_df.columns = ['idf_'+str(i+1) for i in range(tf_df.shape[1]-1)]+ ['涉及告警基站或小区名称']
label = label.merge(tf_df, on='涉及告警基站或小区名称', how='left')

In [6]:
def get_sta(data, col, agg_col):
    temp = data.groupby(['涉及告警基站或小区名称', col]).size().reset_index(name='gaojing_count')
    temp = temp.groupby('涉及告警基站或小区名称')['gaojing_count'].agg(agg_col).reset_index()
    temp.columns = ['涉及告警基站或小区名称'] + [col+'_'+i for i in temp.columns[1:]]
    return temp

In [7]:
agg_col = ['max', 'min', 'mean', 'std', 'median', 'skew',np.ptp]
temp = get_sta(data, '告警标题', agg_col)
label = label.merge(temp, on = '涉及告警基站或小区名称', how= 'left')

In [8]:
label['故障发生时间_min'] = label['故障发生时间'].dt.minute
label['故障发生时间_mon'] = label['故障发生时间'].dt.month
label['故障发生时间_day'] = label['故障发生时间'].dt.day
label['故障发生时间_dayofyear'] = label['故障发生时间'].dt.dayofyear
label['故障发生时间_hour'] = label['故障发生时间'].dt.hour
label['故障发生时间_weekday'] = label['故障发生时间'].dt.weekday
label['故障发生时间_wy'] = label['故障发生时间'].dt.weekofyear
label['故障发生时间_是否周末'] = label['故障发生时间_weekday'].apply(lambda x:1 if x >= 5 else 0)

In [9]:
label['涉及告警基站或小区名称_code'] = preprocessing.LabelEncoder().fit_transform(label['涉及告警基站或小区名称'])
label['涉及告警基站或小区名称_count'] = label.groupby('涉及告警基站或小区名称')['故障发生时间'].transform('count')
temp = label.groupby('涉及告警基站或小区名称')['故障发生时间'].nunique().reset_index(name='涉及告警基站或小区名称_nunique')
label = label.merge(temp, on = '涉及告警基站或小区名称', how= 'left')

In [10]:
label['故障发生时间_day_count'] = label.groupby(['涉及告警基站或小区名称', '故障发生时间_dayofyear'])['工单编号'].transform('count')
label['故障发生时间_week_count'] = label.groupby(['涉及告警基站或小区名称', '故障发生时间_wy'])['工单编号'].transform('count')
label['故障发生时间_hour_count'] = label.groupby(['涉及告警基站或小区名称', '故障发生时间_hour'])['工单编号'].transform('count')
label['故障发生时间_周末_count'] = label.groupby(['涉及告警基站或小区名称', '故障发生时间_是否周末'])['工单编号'].transform('count')
label['故障发生时间_weekday_count'] = label.groupby(['涉及告警基站或小区名称', '故障发生时间_weekday'])['工单编号'].transform('count')
label['故障发生时间_故障发生时间_count'] = label.groupby(['涉及告警基站或小区名称', '故障发生时间'])['工单编号'].transform('count')

In [11]:
pivot = pd.pivot_table(label, index = '涉及告警基站或小区名称', columns= '故障发生时间_hour',values=['工单编号'],aggfunc='count').reset_index().fillna(0)
pivot.columns = ['涉及告警基站或小区名称'] + ['gu'+str(int(i)) for i in pivot['工单编号'].columns.tolist()]
label = label.merge(pivot, on='涉及告警基站或小区名称', how='left')

In [12]:
def get_sta1(data, col, agg_col):
    temp = data[['涉及告警基站或小区名称', col]]
    temp = temp.groupby('涉及告警基站或小区名称')[col].agg(agg_col).reset_index()
    temp.columns = ['涉及告警基站或小区名称'] + [col+'_'+i for i in temp.columns[1:]]
    return temp
agg_col = ['max', 'min', 'mean', 'std', 'median', 'skew',np.ptp]
for i in ['故障发生时间_day_count', '故障发生时间_week_count', '故障发生时间_hour_count',
          '故障发生时间_周末_count', '故障发生时间_weekday_count']:
    temp = get_sta1(label, i, agg_col)
    label = label.merge(temp, on = '涉及告警基站或小区名称', how= 'left')
"""new,故障发生时间_weekday_count"""

'new,故障发生时间_weekday_count'

In [13]:
"""new"""
for i in ['故障发生时间_day_count', '故障发生时间_week_count', '故障发生时间_hour_count',
          '故障发生时间_周末_count', '故障发生时间_weekday_count']:
    label[i+'dmin'] = label[i] - label[i+'_min']

In [14]:
def pivot_df(data):
    pivot = pd.pivot_table(data, index = '涉及告警基站或小区名称', columns= '告警标题',values=['告警发生时间'],aggfunc='count').reset_index().fillna(0)
    pivot.columns = ['涉及告警基站或小区名称'] + pivot['告警发生时间'].columns.tolist()
    gj_index = []
    gj_values = []
    for i,row in pivot.iterrows():
        gj_index.append(row[1:].sort_values(ascending=False).head(4).index.tolist())
        gj_values.append(row[1:].sort_values(ascending=False).head(4).values/row[1:].sum())
    temp = pd.concat([pd.DataFrame(gj_index).astype(int),pd.DataFrame(gj_values)],axis=1)
    temp['涉及告警基站或小区名称'] = pivot['涉及告警基站或小区名称'].values
    return temp
temp = pivot_df(data)
temp.columns = ['top_'+str(i) for i in range(8)]+ ['涉及告警基站或小区名称']
label = label.merge(temp, on='涉及告警基站或小区名称', how='left')

In [15]:
label['小区故障时间_min'] = label.groupby(['涉及告警基站或小区名称'])['故障发生时间_dayofyear'].transform('min')
label['小区故障时间_max'] = label.groupby(['涉及告警基站或小区名称'])['故障发生时间_dayofyear'].transform('max')
label['小区故障时间_ptp'] = label.groupby(['涉及告警基站或小区名称'])['故障发生时间_dayofyear'].transform(np.ptp)
"""new"""
label['故障发生时间_dayofyear_dmin'] = label['故障发生时间_dayofyear'] - label['小区故障时间_min']

In [16]:
def make_feature(data,aggs,name):
    agg_data = data.groupby('涉及告警基站或小区名称').agg(aggs)
    agg_data.columns = agg_data.columns = ['_'.join(i).strip()+name for i in agg_data.columns.tolist()]
    return agg_data.reset_index()
aggs = {
    '故障发生时间_是否周末': ['mean'],
    '故障发生时间_hour': ['nunique'],
    '故障发生时间_weekday': ['nunique'],
    '故障发生时间_day':['nunique'],
    '故障发生时间_wy':['nunique'],
    '故障发生时间_dayofyear':['nunique']
}
agg_df=make_feature(label,aggs,"_故障")
label = label.merge(agg_df, on='涉及告警基站或小区名称', how='left')

In [17]:
label['故障发生时间_int'] = label['故障发生时间'].apply(lambda x: x.value//10**9)
label['故障发生时间_diff'] = label.groupby(['涉及告警基站或小区名称'])['故障发生时间_int'].diff()
label['故障发生时间_diff_min'] = label.groupby(['涉及告警基站或小区名称'])['故障发生时间_diff'].transform('min')
label['故障发生时间_diff_max'] = label.groupby(['涉及告警基站或小区名称'])['故障发生时间_diff'].transform('max')
label['故障发生时间_diff_mean'] = label.groupby(['涉及告警基站或小区名称'])['故障发生时间_diff'].transform('mean')
label['故障发生时间_diff_std'] = label.groupby(['涉及告警基站或小区名称'])['故障发生时间_diff'].transform('std')
label['故障发生时间_diff_mean'] = label.groupby(['涉及告警基站或小区名称'])['故障发生时间_diff'].transform('median')
label['故障发生时间_diff_ptp'] = label.groupby(['涉及告警基站或小区名称'])['故障发生时间_diff'].transform(np.ptp)
del label['故障发生时间_int']

In [18]:
temp = label.groupby(['涉及告警基站或小区名称', '故障发生时间_diff']).size().reset_index(name='diff1')
label = label.merge(temp, on = ['涉及告警基站或小区名称', '故障发生时间_diff'], how= 'left')
agg_col = ['max', 'min', 'mean', 'std', 'median', np.ptp]
temp = get_sta(label, 'diff1', agg_col)
label = label.merge(temp, on = '涉及告警基站或小区名称', how= 'left')

In [19]:
pivot = pd.pivot_table(data, index = '涉及告警基站或小区名称', columns= '告警标题',values=['告警发生时间'],aggfunc='nunique').reset_index().fillna(0)
pivot = pivot.iloc[:,:131].copy()
pivot.columns = ['涉及告警基站或小区名称'] + [i+'n' for i in pivot['告警发生时间'].columns.tolist()]
label = label.merge(pivot, on='涉及告警基站或小区名称', how='left')

In [20]:
data['告警发生时间_min'] = data['告警发生时间'].dt.minute
data['告警发生时间_mon'] = data['告警发生时间'].dt.month
data['告警发生时间_day'] = data['告警发生时间'].dt.day
data['告警发生时间_dayofyear'] = data['告警发生时间'].dt.dayofyear
data['告警发生时间_hour'] = data['告警发生时间'].dt.hour
data['告警发生时间_weekday'] = data['告警发生时间'].dt.weekday
data['告警发生时间_wy'] = data['告警发生时间'].dt.weekofyear
data['告警发生时间_是否周末'] = data['告警发生时间_weekday'].apply(lambda x:1 if x >= 5 else 0)

In [21]:
pivot = pd.pivot_table(data, index = '涉及告警基站或小区名称', columns= '告警发生时间_hour',values=['告警标题'],aggfunc='count').reset_index().fillna(0)
pivot.columns = ['涉及告警基站或小区名称'] + ['gao'+str(int(i)) for i in pivot['告警标题'].columns.tolist()]
label = label.merge(pivot, on='涉及告警基站或小区名称', how='left')

In [22]:
for i in range(24):
    label['gu_gao'+str(i)] = label['gu'+str(i)] / (label['gao'+str(i)]+1)

In [23]:
def make_feature(data,aggs,name):
    agg_data = data.groupby('涉及告警基站或小区名称').agg(aggs)
    agg_data.columns = agg_data.columns = ['_'.join(i).strip()+name for i in agg_data.columns.tolist()]
    return agg_data.reset_index()
aggs = {
    '告警标题': ['count', 'nunique'],
    '告警发生时间_是否周末': ['mean','sum'],
    '告警发生时间_hour': ['nunique'],
    '告警发生时间_weekday': ['nunique'],
    '告警发生时间_day':['nunique'],
    '告警发生时间_wy':['nunique'],
    '告警发生时间_dayofyear':['nunique', 'max', 'min', np.ptp],
}
agg_df=make_feature(data,aggs,"_告警")
label = label.merge(agg_df, on='涉及告警基站或小区名称', how='left')

In [24]:
print(label.shape)
for i in ['_dayofyear', '_wy', '_hour', '_是否周末', '_weekday']:
    haha = data.groupby(['涉及告警基站或小区名称', '告警发生时间'+i])['告警标题'].count().reset_index(name='告警发生时间'+i+'_count')
    haha.columns = ['涉及告警基站或小区名称', '故障发生时间'+i, '告警发生时间'+i+'_count']
    label = label.merge(haha,on = ['涉及告警基站或小区名称', '故障发生时间'+i], how='left')
print(label.shape)
print(label.shape)
for i in ['_dayofyear', '_wy', '_hour', '_是否周末', '_weekday']:
    haha = data.groupby(['涉及告警基站或小区名称', '告警发生时间'+i])['告警标题'].nunique().reset_index(name='告警发生时间'+i+'_nunique')
    haha.columns = ['涉及告警基站或小区名称', '故障发生时间'+i, '告警发生时间'+i+'_nunique']
    label = label.merge(haha,on = ['涉及告警基站或小区名称', '故障发生时间'+i], how='left')
print(label.shape)

(66454, 401)
(66454, 406)
(66454, 406)
(66454, 411)


In [25]:
# def guzhang_time_w2v(label):
#     temp = label[['涉及告警基站或小区名称','故障发生时间_diff']].copy()
#     temp['故障发生时间_diff'] = temp['故障发生时间_diff'].fillna(-1)
#     temp['故障发生时间_diff'] = temp['故障发生时间_diff'].astype(str)
#     sen2 = temp['故障发生时间_diff'].values.tolist()
#     temp = temp.groupby(['涉及告警基站或小区名称'])['故障发生时间_diff'].apply(list).reset_index(name='qwer')
#     temp = temp.sample(frac=1.0, random_state=666)

#     sen = temp['qwer'].values.tolist()
#     try:
#         fastmodel = Word2Vec.load('model_diff.txt')
#     except:
#     #     fastmodel =Word2Vec(sen,size=100,window=8,min_count=1,negative=1,workers=4,iter=10)
#         fastmodel =Word2Vec(sen,size=100,window=8,min_count=1,sg=1,workers=4,iter=10)
#         fastmodel.save('model_diff.txt')
#     w2v = []
#     for i in range(len(sen2)):
#         w2v.append(fastmodel.wv[sen2[i]])
#     del sen;gc.collect()
#     w2v_time_df = pd.DataFrame(w2v)
#     del w2v;gc.collect()
#     w2v_time_df.columns = ['diff_'+str(i+1) for i in w2v_time_df.columns]
#     return  w2v_time_df

In [26]:
# w2v_time_df = guzhang_time_w2v(label)
# label = pd.concat([label, w2v_time_df], axis=1)

In [27]:
label['故障原因定位（大类）'] = label['故障原因定位（大类）'].map(cc_label)
del label['故障发生时间_周末_count_skew']

In [28]:
train = label[~label['故障原因定位（大类）'].isnull()]
test = label[label['故障原因定位（大类）'].isnull()]

In [29]:
col = [i for i in train.columns if i not in ['涉及告警基站或小区名称', '故障发生时间',
                                             '工单编号','告警发生时间',#'故障发生时间_min',
                                             '故障原因定位（大类）']]
X_train = train[col].copy().reset_index(drop=True)
y_train = train['故障原因定位（大类）'].copy().reset_index(drop=True)
X_test = test[col].copy()
print(X_train.shape, X_test.shape)
# (59758, 406) (6696, 406)

(59758, 406) (6696, 406)


In [30]:
# oo = []
# for i in X_train.columns:
#     cc = (X_train[i].value_counts()/X_train.shape[0]).values[0]
#     if cc>0.999:
#         oo.append(i)
# for i in X_test.columns:
#     cc = (X_test[i].value_counts()/X_test.shape[0]).values[0]
#     if cc>0.999:
#         oo.append(i)
# oo = list(set(oo))
# X_train = X_train.drop(columns=oo)
# X_test = X_test.drop(columns=oo)
# print(X_train.shape, X_test.shape)

In [31]:
def accua(df1, df_oof):
    rr = list(np.argsort(df_oof,axis=1)[:,-3:])
    acc = 0
    for i in range(len(df1)):
        acc += len(set([df1.values[i]])&set(list(rr[i])))
    return acc/len(df1)

In [32]:
K = 5
seed = 2018
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
params = {
    'boosting_type': 'gbdt',
    'learning_rate' : 0.04, 
    'verbose': 0,
    'num_leaves':256,
    # 'max_depth':8, 
#     'max_bin':10, 
#     'lambda_l2': 1, 
    'min_child_weight':10,#10
    "num_class":23,
    'objective':'multiclass', 
    'feature_fraction':0.6,#0.4
    'bagging_fraction':0.8, # 0.7
    'bagging_freq':5,  # 3是目前最优的
#           'min_data': 500,
    'seed': 1017,
    'metric':'multi_error',
    'nthread': 14,
    # 'silent': True,
}
# %%time
oof = np.zeros((len(X_train),23))
predictions = np.zeros((len(X_test), 23))
feature_importance_df = pd.DataFrame()
auc_score = []
acc_sc = []
for i, (train_index, val_index) in enumerate(skf.split(X_train,y_train)):
    print("fold {}".format(i))
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    lgb_train = lgb.Dataset(X_tr,y_tr)
    lgb_val = lgb.Dataset(X_val,y_val)
    num_round = 1000
    clf = lgb.train(params, lgb_train, num_round, valid_sets = [lgb_train, lgb_val],#feval=feval_lgb,
                    verbose_eval=100, early_stopping_rounds = 100
                   )
    oof[val_index] = clf.predict(X_val, num_iteration=clf.best_iteration)
    pred = clf.predict(X_val, num_iteration=clf.best_iteration)
    print(roc_auc_score(pd.get_dummies(y_val).values, pred,average='weighted'))
    print(accua(y_val,pred))
    auc_ss = accuracy_score(y_val, np.argmax(pred,axis=1))
    auc_score.append(accua(y_val,pred))
    acc_sc.append(roc_auc_score(pd.get_dummies(y_val).values, pred,average='weighted'))
    print('best iteration = ',clf.best_iteration)
    print('                      auc = ', auc_ss)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = clf.feature_name()
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    predictions += clf.predict(X_test, num_iteration=clf.best_iteration) / skf.n_splits
print(np.mean(auc_score), np.mean(acc_sc))

fold 0
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_error: 0.193046	valid_1's multi_error: 0.301814
[200]	training's multi_error: 0.0931858	valid_1's multi_error: 0.291113
[300]	training's multi_error: 0.0373454	valid_1's multi_error: 0.290026
[400]	training's multi_error: 0.0156704	valid_1's multi_error: 0.289524
Early stopping, best iteration is:
[349]	training's multi_error: 0.0239973	valid_1's multi_error: 0.288605
0.9313388616315693
0.90811805033024
best iteration =  349
                      auc =  0.7113953682802441
fold 1
Training until validation scores don't improve for 100 rounds.
[100]	training's multi_error: 0.191473	valid_1's multi_error: 0.303555
[200]	training's multi_error: 0.0916888	valid_1's multi_error: 0.291677
[300]	training's multi_error: 0.0383658	valid_1's multi_error: 0.290422
Early stopping, best iteration is:
[233]	training's multi_error: 0.0685522	valid_1's multi_error: 0.28967
0.9288821381906036
0.9049769970723547
b

In [ ]:
0.9083466126181813 0.930153187418879

In [38]:
0.9005320595498306 0.9239174690421601
0.9007160395519019 0.9241009818241505
0.9011869335990553 0.9246214461162706
0.9042471634996246 0.9270247590851846
0.9064225839032588 0.9288244537462365
0.9068574812357271 0.9291319478230011
0.9083466126181813 0.930153187418879

SyntaxError: invalid syntax (<ipython-input-38-ca2714fee1d6>, line 1)

In [33]:
col = [i for i in train.columns if i not in ['涉及告警基站或小区名称', '故障发生时间',
                                             '工单编号','告警发生时间',#'故障发生时间_min',
                                             '故障原因定位（大类）']]
X_train = train[col].copy().reset_index(drop=True).values
y_train = train['故障原因定位（大类）'].copy().reset_index(drop=True).values
X_test = test[col].copy().values
print(X_train.shape, X_test.shape)

(59758, 396) (6696, 396)


In [35]:
def accua_xgb(df1, df_oof):
    rr = list(np.argsort(df_oof,axis=1)[:,-3:])
    acc = 0
    for i in range(len(df1)):
        acc += len(set([df1[i]])&set(list(rr[i])))
    return acc/len(df1)

In [38]:
%%time
K = 5
seed = 2021
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
xgb_pred_te_all = 0
xgb_auc_mean = 0
xgb_auc_mean2 = 0
f1 = []
logsocre = []
oof_xgb = np.zeros((len(X_train), 23))
for i, (train_index, test_index) in enumerate(skf.split(X_train,y_train)):
        y_tr, y_val = y_train[train_index].copy(), y_train[test_index].copy()
        X_tr, X_val= X_train[train_index,:].copy(), X_train[test_index,:].copy()
        print( "\nFold ", i+1)

        xgb_tr = xgb.DMatrix(X_tr, y_tr)
        xgb_val = xgb.DMatrix(X_val, y_val)
        xgb_te = xgb.DMatrix(X_test)
        xgb_params = {"objective": 'multi:softprob',
                      "booster" : "gbtree",
                      "eta": 0.06,
                      "max_depth":9,#9
                      "subsample": 0.85,#0.85
                      'eval_metric':'mlogloss',#mlogloss
                      'num_class': 23, 
                      "colsample_bylevel":0.7,#0.8
#                       'gpu_id':0,
                      'tree_method':'gpu_hist',
                      'lambda':6,
                      "seed": 666
                      }
        watchlist = [(xgb_tr, 'train'), (xgb_val, 'eval')]
        xgb_model =xgb.train(xgb_params,
                     xgb_tr,
                     num_boost_round = 2666,
                     evals =watchlist,
                     verbose_eval=200,
                     early_stopping_rounds=100)

        pred = xgb_model.predict(xgb_val, ntree_limit=xgb_model.best_ntree_limit)
        oof_xgb[test_index] = pred
        val_pred = [np.argmax(x) for x in pred]
        acc = accuracy_score(y_val, val_pred)
        logsocre.append(xgb_model.best_score)
        print( "                       acc_score = ", acc )
        print('auc :', roc_auc_score(pd.get_dummies(y_val).values, pred,average='weighted'))
        print('acc_3 :', accua_xgb(y_val, pred))
        f1.append(accua_xgb(y_val, pred))
        print("*"*100)
        pred_te = xgb_model.predict(xgb_te,ntree_limit=xgb_model.best_ntree_limit)
        xgb_pred_te_all = xgb_pred_te_all + pred_te / K
print("="*50+'result'+"="*50)
print( " mean_f1 = ", np.mean(f1) ,np.std(f1))
print( " mean_mlog = ", np.mean(logsocre) ,np.std(logsocre))


Fold  1
[0]	train-merror:0.38176	eval-merror:0.393696
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 100 rounds.
[200]	train-merror:0.136829	eval-merror:0.303068
[400]	train-merror:0.039647	eval-merror:0.295209
Stopping. Best iteration:
[466]	train-merror:0.023391	eval-merror:0.293203

                       acc_score =  0.7067970905442689
auc : 0.9283466282466131
acc_3 : 0.9060279240866148
****************************************************************************************************

Fold  2
[0]	train-merror:0.387068	eval-merror:0.392137
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 100 rounds.


KeyboardInterrupt: 

In [39]:
fold_importance_df.groupby('Feature')['importance'].mean().sort_values(ascending=False)

Feature
故障发生时间_min                 16903
故障发生时间_diff                16802
涉及告警基站或小区名称_code           13000
故障发生时间_dayofyear           11778
故障发生时间_hour                11424
故障发生时间_day                 11297
告警标题_skew                   9348
告警发生时间_是否周末_mean_告警         9320
故障发生时间_diff_mean            8663
故障发生时间_diff_min             8156
故障发生时间_diff_max             7363
告警发生时间_dayofyear_min_告警     6886
idf_19                      6873
故障发生时间_weekday              6857
告警发生时间_dayofyear_ptp_告警     6714
告警标题_std                    6701
小区故障时间_min                  6701
top_5                       6607
告警发生时间_dayofyear_max_告警     6572
小区故障时间_max                  6569
故障发生时间_diff_std             6474
故障发生时间_dayofyear_dmin       6456
top_4                       6079
top_6                       5734
告警标题_mean                   5618
top_1                       5464
top_3                       5435
top_2                       5370
故障发生时间_diff_ptp             5309
idf_71                      5303
  

In [40]:
res=pd.DataFrame(oof,#predictions,
                 columns=['传输系统-传输设备','传输系统-光缆故障','传输系统-其他原因',
                         '动力环境-UPS','动力环境-电力部门供电','动力环境-电源线路故障',
                         '动力环境-动环监控系统','动力环境-动力环境故障','动力环境-高低压设备',
                         '动力环境-环境','动力环境-开关电源','其他-误告警或自动恢复',
                         '人为操作-告警测试','人为操作-工程施工','人为操作-物业原因',
                         '主设备-参数配置异常','主设备-其他','主设备-软件故障',
                         '主设备-设备复位问题','主设备-设备连线故障','主设备-天馈线故障',
                         '主设备-信源问题','主设备-硬件故障'])
res['工单编号'] = train['工单编号'].values
res = res.sort_values('工单编号')

In [41]:
res[['工单编号','传输系统-传输设备','传输系统-光缆故障','传输系统-其他原因',
                         '动力环境-UPS','动力环境-电力部门供电','动力环境-电源线路故障',
                         '动力环境-动环监控系统','动力环境-动力环境故障','动力环境-高低压设备',
                         '动力环境-环境','动力环境-开关电源','其他-误告警或自动恢复',
                         '人为操作-告警测试','人为操作-工程施工','人为操作-物业原因',
                         '主设备-参数配置异常','主设备-其他','主设备-软件故障',
                         '主设备-设备复位问题','主设备-设备连线故障','主设备-天馈线故障',
                         '主设备-信源问题','主设备-硬件故障']].to_csv('../lgb_9068_9299_oof.csv',index=None,encoding='gbk')

In [42]:
res=pd.DataFrame(predictions,#predictions,
                 columns=['传输系统-传输设备','传输系统-光缆故障','传输系统-其他原因',
                         '动力环境-UPS','动力环境-电力部门供电','动力环境-电源线路故障',
                         '动力环境-动环监控系统','动力环境-动力环境故障','动力环境-高低压设备',
                         '动力环境-环境','动力环境-开关电源','其他-误告警或自动恢复',
                         '人为操作-告警测试','人为操作-工程施工','人为操作-物业原因',
                         '主设备-参数配置异常','主设备-其他','主设备-软件故障',
                         '主设备-设备复位问题','主设备-设备连线故障','主设备-天馈线故障',
                         '主设备-信源问题','主设备-硬件故障'])
res['工单编号'] = test['工单编号'].values
res = res.sort_values('工单编号')

In [53]:
res[['工单编号','传输系统-传输设备','传输系统-光缆故障','传输系统-其他原因',
                         '动力环境-UPS','动力环境-电力部门供电','动力环境-电源线路故障',
                         '动力环境-动环监控系统','动力环境-动力环境故障','动力环境-高低压设备',
                         '动力环境-环境','动力环境-开关电源','其他-误告警或自动恢复',
                         '人为操作-告警测试','人为操作-工程施工','人为操作-物业原因',
                         '主设备-参数配置异常','主设备-其他','主设备-软件故障',
                         '主设备-设备复位问题','主设备-设备连线故障','主设备-天馈线故障',
                         '主设备-信源问题','主设备-硬件故障']].to_csv('../submit90689299.csv',index=None,encoding='gbk')

In [43]:
sub = pd.read_csv('../lgb_7036_7077.csv',encoding='gbk')

In [44]:
import scipy.stats as sp
for i in sub.columns:
    if i != '工单编号':
        print(sp.pearsonr(sub[i].values, res[i].values))

(0.9441496200777015, 0.0)
(0.8847655447890934, 0.0)
(0.8289772496761731, 0.0)
(0.9849920825568332, 0.0)
(0.9440950359124413, 0.0)
(0.9244827223200238, 0.0)
(0.8162489788327705, 0.0)
(0.9367740086331099, 0.0)
(0.9211228321392505, 0.0)
(0.9506345566037168, 0.0)
(0.8678439996245684, 0.0)
(0.9880837169738357, 0.0)
(0.9843659180436811, 0.0)
(0.9765332799385134, 0.0)
(0.7030307689021998, 0.0)
(0.9559465932484362, 0.0)
(0.912240542614693, 0.0)
(0.9555448332979691, 0.0)
(0.9409276387850162, 0.0)
(0.9695048908427628, 0.0)
(0.9297998130934574, 0.0)
(0.9539897664614138, 0.0)
(0.9603168194141636, 0.0)
